In [32]:
%run init_script_defines.py

In [3]:
games_trimmed = build_games_trimmed('../Capstone/api/games')

In [6]:
game_shifts = build_game_shifts('../Capstone/api/shifts')

In [ ]:
player_id_set = build_player_id_set(games_trimmed=games_trimmed)
coordinate_shots = build_coordinate_shots(games_trimmed=games_trimmed)
grouped_data = build_grouped_data(games_trimmed=games_trimmed, coordinate_shots=coordinate_shots)
with open('../Capstone/data/player_teams.pkl', 'rb') as f:
    player_teams = pickle.load(f)
team_set = build_team_set(games_trimmed=games_trimmed)
with open('../Capstone/data/team_games.pkl', 'rb') as f:
    team_games = pickle.load(f)
player_id_games = build_player_id_games(games_trimmed=games_trimmed, player_id_set=player_id_set, player_teams=player_teams, team_set=team_set, team_games=team_games)
player_pcts_and_groups=build_player_pcts_and_groups(games_trimmed=games_trimmed, player_id_set=player_id_set, player_teams=player_teams, team_set=team_set, team_games=team_games, player_id_games=player_id_games, coordinate_shots=coordinate_shots, grouped_data=grouped_data)
sadd = build_sadd(games_trimmed=games_trimmed, directory='.', game_shifts=game_shifts, player_id_set=player_id_set, player_teams=player_teams, team_set=team_set, team_games=team_games, player_id_games=player_id_games, coordinate_shots=coordinate_shots, grouped_data=grouped_data, player_pcts_and_groups=player_pcts_and_groups)

build_player_id_set()
build_coordinate_shots()
build_grouped_data()
build_team_set()
build_player_id_games()
build_player_pcts_and_groups()


In [7]:
def build_sadd(games_trimmed=None, directory='.', game_shifts=None, player_id_set=None, player_teams=None, team_set=None, team_games=None, player_id_games=None, coordinate_shots=None, grouped_data=None, player_pcts_and_groups=None):
    if games_trimmed is None:
        print("Please build games_trimmed with the build_games_trimmed() function. \nPass the directory your game .json files are saved in as its argument.")
        return None
    if game_shifts is None:
        game_shifts=build_game_shifts(directory=directory)
    if player_id_set is None:
        player_id_set = build_player_id_set(games_trimmed=games_trimmed)
    if player_teams is None:
        player_teams = build_player_teams(games_trimmed=games_trimmed, player_id_set=player_id_set)
    if team_set is None:
        team_set = build_team_set(games_trimmed=games_trimmed)
    if team_games is None:
        team_games = build_team_games(games_trimmed=games_trimmed, team_set=team_set)
    if player_id_games is None:
        player_id_games = build_player_id_games(games_trimmed=games_trimmed, player_id_set=player_id_set, player_teams=player_teams, team_games=team_games)
    if coordinate_shots is None:
        coordinate_shots = build_coordinate_shots(games_trimmed=games_trimmed)
    if grouped_data is None:
        grouped_data=build_grouped_data(games_trimmed=games_trimmed, coordinate_shots=coordinate_shots)
    if player_pcts_and_groups is None:
        player_pcts_and_groups=build_player_pcts_and_groups(games_trimmed=games_trimmed, player_id_set=player_id_set, player_teams=player_teams, team_set=team_set, team_games=team_games, player_id_games=player_id_games, coordinate_shots=coordinate_shots, grouped_data=grouped_data)
    sadd = {}
    for player in player_id_set:
        sadd[player] = {}
        sadd[player]['pct_diff_total'] = 0
        sadd[player]['sadd_events'] = 0
        sadd[player]['sadd'] = 0
    for gameID, shotID in games_trimmed:
        period, time = get_shot_time(gameID, shotID, games_trimmed=games_trimmed)
        playerList = get_players_on_ice(gameID, period, time, game_shifts=game_shifts)
        shotPct = get_shot_pct(gameID, shotID, games_trimmed=games_trimmed, player_id_set=player_id_set, player_teams=player_teams, team_set=team_set, team_games=team_games, player_id_games=player_id_games, grouped_data=grouped_data, player_pcts_and_groups=player_pcts_and_groups)
        shooterPct = get_shooter_pct(gameID, shotID, games_trimmed=games_trimmed, player_id_set=player_id_set, player_teams=player_teams, team_set=team_set, team_games=team_games, player_id_games=player_id_games, grouped_data=grouped_data, player_pcts_and_groups=player_pcts_and_groups)
        for player in playerList:
            sadd[player]['pct_diff_total'] += (shooterPct - shotPct)
            sadd[player]['sadd_events'] += 1
            sadd[player]['sadd'] = sadd[player]['pct_diff_total']/sadd[player]['sadd_events']
    return sadd

In [8]:
def get_shooter_pct(gameID, shotID, games_trimmed=None, player_id_set=None, player_teams=None, team_set=None, team_games=None, player_id_games=None, coordinate_shots=None, grouped_data=None, player_pcts_and_groups=None):
    if games_trimmed is None:
        print("Please build games_trimmed with the build_games_trimmed() function. \nPass the directory your game .json files are saved in as its argument.")
        return None
    if player_id_set is None:
        player_id_set = build_player_id_set(games_trimmed=games_trimmed)
    if player_teams is None:
        player_teams = build_player_teams(games_trimmed=games_trimmed, player_id_set=player_id_set)
    if team_set is None:
        team_set = build_team_set(games_trimmed=games_trimmed)
    if team_games is None:
        team_games = build_team_games(games_trimmed=games_trimmed, team_set=team_set)
    if player_id_games is None:
        player_id_games = build_player_id_games(games_trimmed=games_trimmed, player_id_set=player_id_set, player_teams=player_teams, team_games=team_games)
    if coordinate_shots is None:
        coordinate_shots = build_coordinate_shots(games_trimmed=games_trimmed)
    if grouped_data is None:
        grouped_data=build_grouped_data(games_trimmed=games_trimmed, coordinate_shots=coordinate_shots)
    if player_pcts_and_groups is None:
        player_pcts_and_groups=build_player_pcts_and_groups(games_trimmed=games_trimmed, player_id_set=player_id_set, player_teams=player_teams, team_set=team_set, team_games=team_games, player_id_games=player_id_games, coordinate_shots=coordinate_shots, grouped_data=grouped_data)
    shooter = get_shot_shooter(gameID, shotID, games_trimmed=games_trimmed)
    return player_pcts_and_groups[shooter][0]

In [9]:
def get_shot_pct(gameID, shotID, games_trimmed=None, player_id_set=None, player_teams=None, team_set=None, team_games=None, player_id_games=None, coordinate_shots=None, grouped_data=None, player_pcts_and_groups=None):
    if games_trimmed is None:
        print("Please build games_trimmed with the build_games_trimmed() function. \nPass the directory your game .json files are saved in as its argument.")
        return None
    if player_id_set is None:
        player_id_set = build_player_id_set(games_trimmed=games_trimmed)
    if player_teams is None:
        player_teams = build_player_teams(games_trimmed=games_trimmed, player_id_set=player_id_set)
    if team_set is None:
        team_set = build_team_set(games_trimmed=games_trimmed)
    if team_games is None:
        team_games = build_team_games(games_trimmed=games_trimmed, team_set=team_set)
    if player_id_games is None:
        player_id_games = build_player_id_games(games_trimmed=games_trimmed, player_id_set=player_id_set, player_teams=player_teams, team_games=team_games)
    if coordinate_shots is None:
        coordinate_shots = build_coordinate_shots(games_trimmed=games_trimmed)
    if grouped_data is None:
        grouped_data=build_grouped_data(games_trimmed=games_trimmed, coordinate_shots=coordinate_shots)
    if player_pcts_and_groups is None:
        player_pcts_and_groups=build_player_pcts_and_groups(games_trimmed=games_trimmed, player_id_set=player_id_set, player_teams=player_teams, team_set=team_set, team_games=team_games, player_id_games=player_id_games, coordinate_shots=coordinate_shots, grouped_data=grouped_data)
    shooter = get_shot_shooter(gameID, shotID, games_trimmed=games_trimmed)
    x, y = get_shot_grouped_coordinates(gameID, shotID, games_trimmed=games_trimmed)
    return player_pcts_and_groups[shooter][1][(x, y)]

In [10]:
def build_player_pcts_and_groups(games_trimmed=None, player_id_set = None, player_teams=None, team_set=None, team_games=None, player_id_games=None, coordinate_shots=None, grouped_data=None):
    if games_trimmed is None:
        print("Please build games_trimmed with the build_games_trimmed() function. \nPass the directory your game .json files are saved in as its argument.")
        return None
    if player_id_set is None:
        player_id_set = build_player_id_set(games_trimmed=games_trimmed)
    if player_teams is None:
        player_teams = build_player_teams(games_trimmed=games_trimmed, player_id_set=player_id_set)
    if team_set is None:
        team_set = build_team_set(games_trimmed=games_trimmed)
    if team_games is None:
        team_games = build_team_games(games_trimmed=games_trimmed, team_set=team_set)
    if player_id_games is None:
        player_id_games = build_player_id_games(games_trimmed=games_trimmed, player_id_set=player_id_set, player_teams=player_teams, team_games=team_games)
    if coordinate_shots is None:
        coordinate_shots = build_coordinate_shots(games_trimmed=games_trimmed)
    if grouped_data is None:
        grouped_data=build_grouped_data(games_trimmed=games_trimmed, coordinate_shots=coordinate_shots)
    player_pcts = {}
    for player in player_id_set:
        player_pcts[player] = get_player_avg_pct_and_groups(player, games_trimmed=games_trimmed, player_id_set=player_id_set, player_teams=player_teams, team_set=team_set, team_games=team_games, player_id_games=player_id_games, grouped_data=grouped_data)
    return player_pcts

In [28]:
grouped_data = build_grouped_data(games_trimmed=games_trimmed, coordinate_shots=coordinate_shots)

build_grouped_data()


In [29]:
type(grouped_data)

NoneType